# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [7]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [8]:
weather_csv_file = "../WeatherPy/output_data/cities.csv"

weather = pd.read_csv(weather_csv_file)

weather.head()

,City,Country,Date,Latitude,Longitude,Cloudiness (%),Humidity (%),Max Temp (F),Wind Speed (mph)
0,barrow,US,2021-10-06 10:06:01,71.2906,-156.7887,90.0,86.0,30.488,6.69
1,albany,US,2021-10-06 10:06:01,42.6001,-73.9662,19.0,92.0,65.588,0.45
2,bredasdorp,ZA,2021-10-06 10:06:02,-34.5322,20.0403,0.0,67.0,70.556,8.05
3,burica,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,atuona,PF,2021-10-06 10:06:02,-9.8000,-139.0333,88.0,79.0,78.602,10.75


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [14]:
gmaps.configure(g_key)

weather = weather.dropna()
humidity = weather['Humidity (%)'].astype(float)

locations = weather[['Latitude', 'Longitude']].astype(float)

fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations,
                                 weights=humidity, 
                                 dissipating=False,
                                 max_intensity=max(weather['Humidity (%)']),
                                 point_radius = 2)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [16]:
filtered_weather = weather

# A max temperature lower than 80 degrees but higher than 70.
filtered_weather = filtered_weather.loc[(filtered_weather['Max Temp (F)'] < 80) & (filtered_weather['Max Temp (F)'] > 70)]

# Wind speed less than 10 mph.
filtered_weather = filtered_weather.loc[filtered_weather['Wind Speed (mph)'] < 10]

# Zero cloudiness.
filtered_weather = filtered_weather.loc[filtered_weather['Cloudiness (%)'] == 0]

# Drop any rows with null values
filtered_weather = filtered_weather.dropna()

filtered_weather

,City,Country,Date,Latitude,Longitude,Cloudiness (%),Humidity (%),Max Temp (F),Wind Speed (mph)
2,bredasdorp,ZA,2021-10-06 10:06:02,-34.5322,20.0403,0.0,67.0,70.556,8.05
92,tadine,NC,2021-10-06 10:06:32,-21.5500,167.8833,0.0,79.0,70.808,5.81
238,guerrero negro,MX,2021-10-06 09:57:52,27.9769,-114.0611,0.0,71.0,71.186,6.86
391,port elizabeth,ZA,2021-10-06 10:08:24,-33.9180,25.5701,0.0,77.0,71.366,8.56
397,dahuk,IQ,2021-10-06 10:08:27,36.8695,42.9940,0.0,20.0,71.276,1.70
404,corinto,GR,2021-10-06 10:08:31,37.9407,22.9573,0.0,47.0,74.678,3.02
446,bardiyah,LY,2021-10-06 10:08:45,31.7561,25.0865,0.0,52.0,74.948,4.20
510,gat,IL,2021-10-06 10:07:06,31.6100,34.7642,0.0,47.0,79.754,6.13
549,nagato,JP,2021-10-06 10:09:26,34.3833,131.2000,0.0,85.0,72.464,0.69
570,touho,NC,2021-10-06 10:09:33,-20.7833,165.2333,0.0,74.0,72.410,2.56


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [27]:
hotel_df = filtered_weather

# params dictionary to update each iteration
params = {
    'radius': 5000,
    'types': 'lodging',
    'key': g_key
}


for index, row in hotel_df.iterrows():
    Lat = row["Latitude"]
    Lng = row["Longitude"]

    params['location'] = f'{Lat},{Lng}'
    
    base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'

    print(f'Retrieving Results for Index {index}: {row["City"]}.')
    response = requests.get(base_url, params=params).json()
 
    
    results = response['results']
    
    try:
        print(f'Closest hotel is {results[0]["name"]}.')
        hotel_df.loc[index, 'Hotel Name'] = results[0]["name"]
        
    except (KeyError, IndexError):
        print('Missing field/result... skipping.')
        
    print('------------')

Retrieving Results for Index 2: bredasdorp.
Closest hotel is Bredasdorp Country Manor.
------------
Retrieving Results for Index 92: tadine.
Missing field/result... skipping.
------------
Retrieving Results for Index 238: guerrero negro.
Closest hotel is Casa Laguna, Bed & Breakfast.
------------
Retrieving Results for Index 391: port elizabeth.
Closest hotel is 39 On Nile Guest House.
------------
Retrieving Results for Index 397: dahuk.
Closest hotel is Parwar Hotel.
------------
Retrieving Results for Index 404: corinto.
Closest hotel is Ephira Hotel.
------------
Retrieving Results for Index 446: bardiyah.
Closest hotel is بورتو بردية.
------------
Retrieving Results for Index 510: gat.
Closest hotel is OlusHome.
------------
Retrieving Results for Index 549: nagato.
Closest hotel is いそう庵.
------------
Retrieving Results for Index 570: touho.
Closest hotel is Auberge Timay Evasion.
------------
Retrieving Results for Index 604: concordia.
Closest hotel is Palmar Hotel Casino.
-----

In [34]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [35]:
# Add marker layer ontop of heat map
# Add the layer to the map

hotel_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content=hotel_info
)

fig.add_layer(markers)
fig.add_layer(hotel_layer)
fig



Figure(layout=FigureLayout(height='420px'))